# Setup

In [98]:
%matplotlib inline
from pylab import *

import detectorgeometry_trap as geom_trap
reload(geom_trap)

import decay_properties as decay
reload(decay)

<module 'decay_properties' from 'decay_properties.py'>

In [99]:
# example data input to class to determine effective area
theta_src = radians(10) #radians
h = 35.5 #km
R = 6356. # km
ice =4
theta_view = radians(1) #degrees
N = 200000 # number of samples
E_nu = 19 #log energy in eV of incident neutrino 
E_cut = 14

geom_area = geom_trap.Area(theta_src,h,R,theta_view, N)


In [100]:
A_deg, phi_e, theta_e, view_angle, exit_angle, norm, e_dot = geom_area.event_retention()

In [101]:
print A_deg

1.2634566904401228


In [102]:
print min(degrees(exit_angle))
print norm

9.050735786699155
[35.99647146 36.13191243 36.1582675  ... 35.97797576 36.01083939
 35.97640283]


In [103]:
# Assign to each event a tau energy, tau decay point, a decay probability, calculated view angle
# inputs : theta_src, ice, E_cut, e_dot, phi_e, t_e, rho, E_nu,h,R,view_cut, exit_angles
decay1 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, norm,E_nu, h, R, theta_view, exit_angle)
                        

9.361975901020198


In [104]:
print decay1.E_tau()

-80.44620446835935


IndexError: index 135 is out of bounds for axis 0 with size 135

In [71]:
ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_decay_dist, ret_p_decay ,ret_view_angle, ret_fractions,ret_types = decay1.event_retention()
#print ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_decay_dist, ret_p_decay ,ret_view_angle
#print decay1.degree_eff_area()

[0. 0. 0. ... 0. 0. 0.]


TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'NoneType'